In [1]:
from Skeleton import Skeleton,DynamicSkeleton,ConstrainedSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time

s12 = Skeleton('BODY12.xml')
s15 = ConstrainedSkeleton('BODY15_constrained_3D.xml')
# Rajagopal
# s = DynamicSkeleton('BODY15_constrained_3D.xml')
# BSM
kps =  ['RKnee', 'LWrist', 'RHip', 'RShoulder',  'LElbow', 'LHip', 'RElbow', 'RWrist', 'LKnee', 'LShoulder', 'RAnkle', 'LAnkle']
kps_ice = ['right_knee', 'left_wrist', 'right_hip', 'right_shoulder', 'left_elbow', 'left_hip', 'right_elbow', 'right_wrist', 'left_knee', 'left_shoulder', 'right_ankle','left_ankle']
gui = nimble.NimbleGUI()
gui.serve(8080)
# sudo lsof -iTCP:8080 -sTCP:LISTEN
with open('tmp/camera-parameters.json') as f:
    parameters = json.load(f)


GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070
Web GUI serving on http://localhost:8080


In [2]:
# Qualitative analysis from ICE lab test case
import json
filename = "/home/rmhri/mrom/data/ice/interaction_1p/aggregator_source_flk.flk.json"
with open(filename) as f:
    file = json.load(f)

people = {}
last_seen = {}

outfile = file.copy()

for f in range(len(file)):
    print("frame",f)
    frame = file[f]
    for p in range(len(frame["track_ids"])):
        data = frame["kp3d"][p]
        
        # If there is a new id, then add a dynamic skeleton instance
        if frame["track_ids"][p] not in people.keys():
            print("new body found:",frame["track_ids"][p])
            people[frame["track_ids"][p]] = DynamicSkeleton(config='BODY15_constrained_3D.xml', osim_file='/media/rmhri/mrom/data/skel_models_v1.1/bsm.osim', max_velocity=2)
            dt = 100
        else:
            dt = (frame["timestamp"]-last_seen[frame["track_ids"][p]])/1000
            # print("dt:",dt)

        # Extract the 12 keypoints 
        target = []
        for k in kps_ice:
            if k in data.keys():
                target.append([data[k][0],data[k][1],data[k][2]])
            else:
                target.append([np.nan,np.nan,np.nan])
        target = np.array(target)
        # print(target)
        
            
        
        s12.load_from_numpy(target,kps)
        people[frame["track_ids"][p]].load_from_BODY12(s12)
        people[frame["track_ids"][p]].estimate_confidence()
        people[frame["track_ids"][p]].scale()
        
        print([str(j.name)+": "+str(j.confidence) for j in people[frame["track_ids"][p]].keypoints_list])
        print(people[frame["track_ids"][p]]._nimble.getBodyScales().reshape(-1,3)[:,0])
        
        people[frame["track_ids"][p]].filter(dt=dt,Q=0.01)
        
        last_seen[frame["track_ids"][p]] = frame["timestamp"]
        gui.nativeAPI().renderSkeleton(people[frame["track_ids"][p]]._nimble)
        
        # Save joints position to a new json
        pos = people[frame["track_ids"][p]]._nimble.getJointWorldPositions(people[frame["track_ids"][p]].joints)
        pos = people[frame["track_ids"][p]].correct(pos)

        # DEBUG
        s12.load_from_numpy(pos.reshape(-1,3),kps)
        s15.load_from_BODY12(s12)


        outfile[f]["kp3d"].append(file[f]["kp3d"][p].copy())
        outfile[f]["kp2d"].append(file[f]["kp2d"][p].copy())
        outfile[f]["track_ids"].append(file[f]["track_ids"][p]+10)
        outfile[f]["kp3d"][-1] = {}
        # print([ file[f]["track_ids"][p1] for p1 in range(len(file[f]["track_ids"]))] )
        
        
        for k in range(len(kps_ice)):      
            outfile[f]["kp3d"][-1][kps_ice[k]] = [np.nan,np.nan,np.nan]
            outfile[f]["kp3d"][-1][kps_ice[k]][0] = pos[3*k]
            outfile[f]["kp3d"][-1][kps_ice[k]][1] = pos[3*k+1]
            outfile[f]["kp3d"][-1][kps_ice[k]][2] = pos[3*k+2]
        

frame 0
frame 1
frame 2
frame 3
frame 4
frame 5
frame 6
frame 7
frame 8
frame 9
frame 10
frame 11
frame 12
frame 13
frame 14
frame 15
frame 16
frame 17
frame 18
frame 19
frame 20
frame 21
frame 22
frame 23
frame 24
frame 25
frame 26
frame 27
frame 28
frame 29
frame 30
frame 31
frame 32
frame 33
frame 34
frame 35
frame 36
frame 37
frame 38
frame 39
frame 40
frame 41
frame 42
frame 43
frame 44
frame 45
frame 46
frame 47
frame 48
frame 49
frame 50
new body found: 1
Setting len to 0.1
Setting neutral pos to   0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0
symmetric law for femur_l
symmetric law for tibia_l
['Root: 0.8', 'MidShoulder: 0.7000000000000001', 'LShoulder: 0.8', 'LElbow: 0.8', 'LWrist: 0.9', 'RShoulder: 0.8', 'RElbow: 0.8', 'RWrist: 0.9', 'MidHip: 1', 'LHip: 0.9', 'LKnee: 0.8', 'LAnkle: 0.9', 'RHip: 0.9', 'RKnee: 0.8', 'RAnkle: 0.9']
[1.05623777 0.95623777 0.95623777 1.05623777 1.05623777 1.05623777
 0.9562

/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/Skeleton/DynamicSkeleton.py:263: RuntimeWarning: Mean of empty slice
  sc = np.nanmean(self.bones_dict[self.body_dict[b]].history) / self.skeleton_from_nimble.bones_dict[self.body_dict[b]].length
/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/Skeleton/DynamicSkeleton.py:267: RuntimeWarning: Mean of empty slice
  sc = np.nanmean(self.bones_dict[self.symmetry[self.body_dict[b]]].history) / self.skeleton_from_nimble.bones_dict[self.symmetry[self.body_dict[b]]].length


frame 51
symmetric law for femur_l
symmetric law for tibia_l
['Root: 0.8', 'MidShoulder: 0.7000000000000001', 'LShoulder: 0.8', 'LElbow: 0.8', 'LWrist: 0.9', 'RShoulder: 0.8', 'RElbow: 0.8', 'RWrist: 0.9', 'MidHip: 1', 'LHip: 0.9', 'LKnee: 0.8', 'LAnkle: 0.9', 'RHip: 0.9', 'RKnee: 0.8', 'RAnkle: 0.9']
[1.01379485 0.91379485 0.91379485 1.01379485 1.01379485 1.01379485
 0.91379485 0.91379485 1.01379485 1.01379485 1.01379485 1.01379485
 1.01379485 1.01379485 1.01379485 1.01379485 0.91379485 0.91379485
 1.01379485 1.01379485 1.01379485 0.91379485 0.91379485 1.01379485]


/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/Skeleton/DynamicSkeleton.py:269: RuntimeWarning: Mean of empty slice
  sc_sym = np.nanmean(self.bones_dict[self.symmetry[self.body_dict[b]]].history) / self.skeleton_from_nimble.bones_dict[self.symmetry[self.body_dict[b]]].length


frame 52
symmetric law for femur_r
symmetric law for tibia_l
['Root: 0.8', 'MidShoulder: 0.7000000000000001', 'LShoulder: 1.0', 'LElbow: 0.9', 'LWrist: 0.9', 'RShoulder: 0.8', 'RElbow: 0.8', 'RWrist: 0.9', 'MidHip: 1', 'LHip: 0.9', 'LKnee: 0.8', 'LAnkle: 0.9', 'RHip: 0.9', 'RKnee: 0.8', 'RAnkle: 0.9']
[1.0050438  0.9050438  0.9050438  1.0050438  1.0050438  1.0050438
 0.9050438  0.9050438  1.0050438  1.0050438  1.0050438  1.0050438
 1.0050438  1.0050438  1.0050438  0.99197544 0.9050438  0.9050438
 1.0050438  1.0050438  0.99197544 0.9050438  0.9050438  1.0050438 ]
frame 53
symmetric law for femur_r
symmetric law for tibia_l
['Root: 0.8', 'MidShoulder: 0.7000000000000001', 'LShoulder: 1.0', 'LElbow: 1', 'LWrist: 1', 'RShoulder: 0.8', 'RElbow: 0.8', 'RWrist: 0.9', 'MidHip: 1', 'LHip: 0.9', 'LKnee: 0.8', 'LAnkle: 0.9', 'RHip: 0.9', 'RKnee: 1.0', 'RAnkle: 1']
[1.00193927 0.90193927 0.90193927 1.00193927 1.00193927 1.00193927
 0.90193927 0.90193927 1.00193927 1.00193927 1.00193927 1.00193927


In [3]:
# Read the json and dump
outfilename = "tmp/aggregator_source_flk.flk.bm.json"
with open(outfilename, 'w') as f:
  json.dump(outfile, f)